In [1]:
import requests
import pandas as pd
import house_finder
import time
import googlemaps
import numpy as np
import pickle as pkl
API_KEY = "YOUR-KEY-HERE"
G_API_KEY = 'YOUR-KEY-HERE'
BASE_URL = "https://api.rentcast.io/v1/listings/rental/long-term"
headers = {
    "X-Api-Key": API_KEY
}

In [2]:
def get_lat_lng(address, api_key):

    geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(geocode_url)
    
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            lat = data['results'][0]['geometry']['location']['lat']
            lng = data['results'][0]['geometry']['location']['lng']
            return lat, lng
        else:
            print("Error: No results found.")
            return None, None
    else:
        print(f"Error: {response.status_code}")
        return None, None

address = "1600 Amphitheatre Parkway, Mountain View, CA"
latitude, longitude = get_lat_lng(address, G_API_KEY)

print(f"Latitude: {latitude}, Longitude: {longitude}")


Latitude: 37.4225078, Longitude: -122.0847365


In [3]:
def get_rental_listings(params):

    response = requests.get(BASE_URL, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve data. Error: {response.status_code} - {response.text}")


In [4]:
params = {
    "latitude": latitude,
    "longitude": longitude,
    'radius': 2000,
    'limit': 100
}
listings = get_rental_listings(params)
print(listings)


[{'id': '7584-Red-Willow-St,-Sacramento,-CA-95822', 'formattedAddress': '7584 Red Willow St, Sacramento, CA 95822', 'addressLine1': '7584 Red Willow St', 'addressLine2': None, 'city': 'Sacramento', 'state': 'CA', 'zipCode': '95822', 'county': 'Sacramento', 'latitude': 38.482083, 'longitude': -121.499091, 'propertyType': 'Single Family', 'bedrooms': 3, 'bathrooms': 2, 'squareFootage': 1184, 'status': 'Active', 'price': 2150, 'listingType': 'Standard', 'listedDate': '2024-12-23T00:00:00.000Z', 'removedDate': None, 'createdDate': '2024-12-10T00:00:00.000Z', 'lastSeenDate': '2024-12-23T04:48:46.057Z', 'daysOnMarket': 1, 'history': {'2024-12-23': {'event': 'Rental Listing', 'price': 2150, 'listingType': 'Standard', 'listedDate': '2024-12-23T00:00:00.000Z', 'removedDate': None, 'daysOnMarket': 1}}}, {'id': '1609-Alligator-Aly,-Sacramento,-CA-95816', 'formattedAddress': '1609 Alligator Aly, Sacramento, CA 95816', 'addressLine1': '1609 Alligator Aly', 'addressLine2': None, 'city': 'Sacramento'

In [5]:

df=pd.DataFrame(listings)
df

,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,longitude,...,status,price,listingType,listedDate,removedDate,createdDate,lastSeenDate,daysOnMarket,history,yearBuilt
0,"7584-Red-Willow-St,-Sacramento,-CA-95822","7584 Red Willow St, Sacramento, CA 95822",7584 Red Willow St,None,Sacramento,CA,95822,Sacramento,38.482083,-121.499091,...,Active,2150,Standard,2024-12-23T00:00:00.000Z,None,2024-12-10T00:00:00.000Z,2024-12-23T04:48:46.057Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN
1,"1609-Alligator-Aly,-Sacramento,-CA-95816","1609 Alligator Aly, Sacramento, CA 95816",1609 Alligator Aly,None,Sacramento,CA,95816,Sacramento,38.562727,-121.460786,...,Active,1900,Standard,2024-12-23T00:00:00.000Z,None,2024-10-19T00:00:00.000Z,2024-12-23T04:48:46.056Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN
2,"4528-10th-Ave,-Sacramento,-CA-95820","4528 10th Ave, Sacramento, CA 95820",4528 10th Ave,None,Sacramento,CA,95820,Sacramento,38.542584,-121.453691,...,Active,2295,Standard,2024-12-23T00:00:00.000Z,None,2024-09-20T00:00:00.000Z,2024-12-23T04:48:44.759Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN
3,"8709-Sangria-Ct,-Elk-Grove,-CA-95624","8709 Sangria Ct, Elk Grove, CA 95624",8709 Sangria Ct,None,Elk Grove,CA,95624,Sacramento,38.421390,-121.379612,...,Active,2299,Standard,2024-12-23T00:00:00.000Z,None,2024-11-21T00:00:00.000Z,2024-12-23T04:48:44.759Z,1,"{'2024-11-30': {'event': 'Rental Listing', 'pr...",NaN
4,"9424-Winewood-Cir,-Elk-Grove,-CA-95758","9424 Winewood Cir, Elk Grove, CA 95758",9424 Winewood Cir,None,Elk Grove,CA,95758,Sacramento,38.414201,-121.404665,...,Active,2695,Standard,2024-12-23T00:00:00.000Z,None,2024-12-11T00:00:00.000Z,2024-12-23T04:48:44.757Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"9444-Harbour-Point-Dr,-Apt-32,-Elk-Grove,-CA-9...","9444 Harbour Point Dr, Apt 32, Elk Grove, CA 9...",9444 Harbour Point Dr,Apt 32,Elk Grove,CA,95758,Sacramento,38.412856,-121.478759,...,Active,2735,Standard,2024-12-23T00:00:00.000Z,None,2024-12-14T00:00:00.000Z,2024-12-23T04:48:13.002Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN
96,"731-16th-St,-Apt-107,-Sacramento,-CA-95814","731 16th St, Apt 107, Sacramento, CA 95814",731 16th St,Apt 107,Sacramento,CA,95814,Sacramento,38.580635,-121.483734,...,Active,1615,Standard,2024-12-23T00:00:00.000Z,None,2023-08-21T00:00:00.000Z,2024-12-23T04:48:12.998Z,1,"{'2024-10-21': {'event': 'Rental Listing', 'pr...",NaN
97,"7459-Rush-River-Dr,---90,-Sacramento,-CA-95831","7459 Rush River Dr, # 90, Sacramento, CA 95831",7459 Rush River Dr,# 90,Sacramento,CA,95831,Sacramento,38.487198,-121.534485,...,Active,2249,Standard,2024-12-23T00:00:00.000Z,None,2024-10-26T00:00:00.000Z,2024-12-23T04:48:12.997Z,1,"{'2024-12-07': {'event': 'Rental Listing', 'pr...",NaN
98,"7952-Pocket-Rd,---370,-Sacramento,-CA-95831","7952 Pocket Rd, # 370, Sacramento, CA 95831",7952 Pocket Rd,# 370,Sacramento,CA,95831,Sacramento,38.476185,-121.532822,...,Active,1895,Standard,2024-12-23T00:00:00.000Z,None,2024-11-16T00:00:00.000Z,2024-12-23T04:48:12.997Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN


In [6]:
df[df['status']=='Active']

,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,longitude,...,status,price,listingType,listedDate,removedDate,createdDate,lastSeenDate,daysOnMarket,history,yearBuilt
0,"7584-Red-Willow-St,-Sacramento,-CA-95822","7584 Red Willow St, Sacramento, CA 95822",7584 Red Willow St,None,Sacramento,CA,95822,Sacramento,38.482083,-121.499091,...,Active,2150,Standard,2024-12-23T00:00:00.000Z,None,2024-12-10T00:00:00.000Z,2024-12-23T04:48:46.057Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN
1,"1609-Alligator-Aly,-Sacramento,-CA-95816","1609 Alligator Aly, Sacramento, CA 95816",1609 Alligator Aly,None,Sacramento,CA,95816,Sacramento,38.562727,-121.460786,...,Active,1900,Standard,2024-12-23T00:00:00.000Z,None,2024-10-19T00:00:00.000Z,2024-12-23T04:48:46.056Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN
2,"4528-10th-Ave,-Sacramento,-CA-95820","4528 10th Ave, Sacramento, CA 95820",4528 10th Ave,None,Sacramento,CA,95820,Sacramento,38.542584,-121.453691,...,Active,2295,Standard,2024-12-23T00:00:00.000Z,None,2024-09-20T00:00:00.000Z,2024-12-23T04:48:44.759Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN
3,"8709-Sangria-Ct,-Elk-Grove,-CA-95624","8709 Sangria Ct, Elk Grove, CA 95624",8709 Sangria Ct,None,Elk Grove,CA,95624,Sacramento,38.421390,-121.379612,...,Active,2299,Standard,2024-12-23T00:00:00.000Z,None,2024-11-21T00:00:00.000Z,2024-12-23T04:48:44.759Z,1,"{'2024-11-30': {'event': 'Rental Listing', 'pr...",NaN
4,"9424-Winewood-Cir,-Elk-Grove,-CA-95758","9424 Winewood Cir, Elk Grove, CA 95758",9424 Winewood Cir,None,Elk Grove,CA,95758,Sacramento,38.414201,-121.404665,...,Active,2695,Standard,2024-12-23T00:00:00.000Z,None,2024-12-11T00:00:00.000Z,2024-12-23T04:48:44.757Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"9444-Harbour-Point-Dr,-Apt-32,-Elk-Grove,-CA-9...","9444 Harbour Point Dr, Apt 32, Elk Grove, CA 9...",9444 Harbour Point Dr,Apt 32,Elk Grove,CA,95758,Sacramento,38.412856,-121.478759,...,Active,2735,Standard,2024-12-23T00:00:00.000Z,None,2024-12-14T00:00:00.000Z,2024-12-23T04:48:13.002Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN
96,"731-16th-St,-Apt-107,-Sacramento,-CA-95814","731 16th St, Apt 107, Sacramento, CA 95814",731 16th St,Apt 107,Sacramento,CA,95814,Sacramento,38.580635,-121.483734,...,Active,1615,Standard,2024-12-23T00:00:00.000Z,None,2023-08-21T00:00:00.000Z,2024-12-23T04:48:12.998Z,1,"{'2024-10-21': {'event': 'Rental Listing', 'pr...",NaN
97,"7459-Rush-River-Dr,---90,-Sacramento,-CA-95831","7459 Rush River Dr, # 90, Sacramento, CA 95831",7459 Rush River Dr,# 90,Sacramento,CA,95831,Sacramento,38.487198,-121.534485,...,Active,2249,Standard,2024-12-23T00:00:00.000Z,None,2024-10-26T00:00:00.000Z,2024-12-23T04:48:12.997Z,1,"{'2024-12-07': {'event': 'Rental Listing', 'pr...",NaN
98,"7952-Pocket-Rd,---370,-Sacramento,-CA-95831","7952 Pocket Rd, # 370, Sacramento, CA 95831",7952 Pocket Rd,# 370,Sacramento,CA,95831,Sacramento,38.476185,-121.532822,...,Active,1895,Standard,2024-12-23T00:00:00.000Z,None,2024-11-16T00:00:00.000Z,2024-12-23T04:48:12.997Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN


In [7]:
df.sort_values(by='price',inplace=True)

In [8]:
df

,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,longitude,...,status,price,listingType,listedDate,removedDate,createdDate,lastSeenDate,daysOnMarket,history,yearBuilt
18,"1819-G-St,-Sacramento,-CA-95811","1819 G St, Sacramento, CA 95811",1819 G St,None,Sacramento,CA,95811,Sacramento,38.580795,-121.480276,...,Active,1295,Standard,2024-12-23T00:00:00.000Z,None,2024-03-10T00:00:00.000Z,2024-12-23T04:48:44.681Z,1,"{'2024-10-21': {'event': 'Rental Listing', 'pr...",NaN
70,"4359-Shining-Star-Dr,-Apt-4,-Sacramento,-CA-95823","4359 Shining Star Dr, Apt 4, Sacramento, CA 95823",4359 Shining Star Dr,Apt 4,Sacramento,CA,95823,Sacramento,38.487406,-121.457061,...,Active,1495,Standard,2024-12-23T00:00:00.000Z,None,2021-11-27T02:16:44.578Z,2024-12-23T04:48:13.087Z,1,"{'2024-11-21': {'event': 'Rental Listing', 'pr...",NaN
89,"3225-Julliard-Dr,-Apt-338,-Sacramento,-CA-95826","3225 Julliard Dr, Apt 338, Sacramento, CA 95826",3225 Julliard Dr,Apt 338,Sacramento,CA,95826,Sacramento,38.548969,-121.393341,...,Active,1545,Standard,2024-12-23T00:00:00.000Z,None,2024-12-12T00:00:00.000Z,2024-12-23T04:48:13.015Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN
81,"6050-S-Land-Park-Dr,-Sacramento,-CA-95822","6050 S Land Park Dr, Sacramento, CA 95822",6050 S Land Park Dr,None,Sacramento,CA,95822,Sacramento,38.517521,-121.514732,...,Active,1550,Standard,2024-12-23T00:00:00.000Z,None,2024-01-22T00:00:00.000Z,2024-12-23T04:48:13.030Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN
96,"731-16th-St,-Apt-107,-Sacramento,-CA-95814","731 16th St, Apt 107, Sacramento, CA 95814",731 16th St,Apt 107,Sacramento,CA,95814,Sacramento,38.580635,-121.483734,...,Active,1615,Standard,2024-12-23T00:00:00.000Z,None,2023-08-21T00:00:00.000Z,2024-12-23T04:48:12.998Z,1,"{'2024-10-21': {'event': 'Rental Listing', 'pr...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,"9278-Elberon-Way,-Elk-Grove,-CA-95758","9278 Elberon Way, Elk Grove, CA 95758",9278 Elberon Way,None,Elk Grove,CA,95758,Sacramento,38.420070,-121.478188,...,Active,3600,Standard,2024-12-23T00:00:00.000Z,None,2020-04-30T13:07:48.139Z,2024-12-23T04:48:44.645Z,1,"{'2024-10-26': {'event': 'Rental Listing', 'pr...",NaN
64,"8609-Aubrey-Ct,-Elk-Grove,-CA-95624","8609 Aubrey Ct, Elk Grove, CA 95624",8609 Aubrey Ct,None,Elk Grove,CA,95624,Sacramento,38.444808,-121.369831,...,Active,3780,Standard,2024-12-23T00:00:00.000Z,None,2024-12-19T00:00:00.000Z,2024-12-23T04:48:44.615Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN
20,"404-17th-St,-Sacramento,-CA-95811","404 17th St, Sacramento, CA 95811",404 17th St,None,Sacramento,CA,95811,Sacramento,38.584159,-121.481331,...,Active,3800,Standard,2024-12-23T00:00:00.000Z,None,2024-12-20T00:00:00.000Z,2024-12-23T04:48:44.671Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN
22,"1861-10th-Ave,-Sacramento,-CA-95818","1861 10th Ave, Sacramento, CA 95818",1861 10th Ave,None,Sacramento,CA,95818,Sacramento,38.545555,-121.494222,...,Active,5495,Standard,2024-12-23T00:00:00.000Z,None,2024-08-20T00:00:00.000Z,2024-12-23T04:48:44.668Z,1,"{'2024-10-07': {'event': 'Rental Listing', 'pr...",NaN


In [9]:
price_df = df

In [10]:
price_df = price_df.rename(columns={'index': 'price_id'})

In [11]:
df_homes = price_df[['latitude','longitude']]
df_homes

,latitude,longitude
18,38.580795,-121.480276
70,38.487406,-121.457061
89,38.548969,-121.393341
81,38.517521,-121.514732
96,38.580635,-121.483734
...,...,...
38,38.420070,-121.478188
64,38.444808,-121.369831
20,38.584159,-121.481331
22,38.545555,-121.494222


In [12]:
houses = [
    {"id": idx, "location": (row['latitude'], row['longitude'])}
    for idx, row in df_homes.iterrows()
]

In [13]:
houses

[{'id': 18, 'location': (38.580795, -121.480276)},
 {'id': 70, 'location': (38.487406, -121.457061)},
 {'id': 89, 'location': (38.548969, -121.393341)},
 {'id': 81, 'location': (38.517521, -121.514732)},
 {'id': 96, 'location': (38.580635, -121.483734)},
 {'id': 53, 'location': (38.53851, -121.470828)},
 {'id': 86, 'location': (38.599693, -121.490906)},
 {'id': 99, 'location': (38.576633, -121.502159)},
 {'id': 24, 'location': (38.537403, -121.469059)},
 {'id': 35, 'location': (38.549133, -121.508858)},
 {'id': 5, 'location': (38.413945, -121.479663)},
 {'id': 78, 'location': (38.598305, -121.501144)},
 {'id': 98, 'location': (38.476185, -121.532822)},
 {'id': 1, 'location': (38.562727, -121.4607865)},
 {'id': 45, 'location': (38.414203, -121.379111)},
 {'id': 31, 'location': (38.531158, -121.518375)},
 {'id': 72, 'location': (38.55004, -121.473351)},
 {'id': 14, 'location': (38.539777, -121.448385)},
 {'id': 93, 'location': (38.487198, -121.534485)},
 {'id': 0, 'location': (38.482083,

In [14]:
with open('houses.pkl','wb') as f:
    pkl.dump(houses,f)

In [16]:
gmaps = googlemaps.Client(key=G_API_KEY)

In [17]:
start_time = time.time()
print(house_finder.top_houses_by_average(houses,10)) #can include radius as the third argument
end_time = time.time()
print("total time using average distance: ",end_time-start_time)

[{'House_ID': 60, 'distance': 0.1293076664968248}, {'House_ID': 67, 'distance': 0.17292803260964124}, {'House_ID': 39, 'distance': 0.20008202120123775}, {'House_ID': 46, 'distance': 0.24332956367206435}, {'House_ID': 14, 'distance': 0.2668795903910921}, {'House_ID': 48, 'distance': 0.26790485540656417}, {'House_ID': 79, 'distance': 0.2715088172791332}, {'House_ID': 41, 'distance': 0.2717884350106255}, {'House_ID': 66, 'distance': 0.28222749698634225}, {'House_ID': 99, 'distance': 0.2909888525731045}]
total time using average distance:  115.8410530090332


In [18]:
start_time = time.time()
print(house_finder.top_houses_by_weighted(houses,10)) #can include radius as the third argument
end_time = time.time()
print("total time using weighted average distance: ",end_time-start_time)

[{'House_ID': 60, 'distance': 0.10180606545198186}, {'House_ID': 67, 'distance': 0.1592710392123856}, {'House_ID': 39, 'distance': 0.17350581361519524}, {'House_ID': 31, 'distance': 0.1895903432752378}, {'House_ID': 99, 'distance': 0.19169251004129875}, {'House_ID': 79, 'distance': 0.19239989035776708}, {'House_ID': 17, 'distance': 0.2007105518362299}, {'House_ID': 46, 'distance': 0.20452309600892024}, {'House_ID': 91, 'distance': 0.21256853094607275}, {'House_ID': 90, 'distance': 0.21294220033368477}]
total time using weighted average distance:  86.74255108833313


In [19]:
start_time = time.time()
print(house_finder.top_houses_by_knn(houses,10)) #can include radius as the third argument
end_time = time.time()
print("total time using K-NN: ",end_time-start_time)

[{'House_ID': 60, 'distance': 0.021540921537193305}, {'House_ID': 17, 'distance': 0.049088446195334735}, {'House_ID': 99, 'distance': 0.05157393714193396}, {'House_ID': 31, 'distance': 0.054680800825183}, {'House_ID': 43, 'distance': 0.054680800825183}, {'House_ID': 66, 'distance': 0.06441564036602997}, {'House_ID': 79, 'distance': 0.06938662225922843}, {'House_ID': 39, 'distance': 0.08139982850112469}, {'House_ID': 67, 'distance': 0.08471381642992366}, {'House_ID': 87, 'distance': 0.08678505888542302}]
total time using K-NN:  87.4808452129364


## Post evaluation after looking at results from Google-Maps code

In [24]:
price_df[price_df.index==60]

,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,longitude,...,status,price,listingType,listedDate,removedDate,createdDate,lastSeenDate,daysOnMarket,history,yearBuilt
60,"2800-F-St,-Sacramento,-CA-95816","2800 F St, Sacramento, CA 95816",2800 F St,None,Sacramento,CA,95816,Sacramento,38.577953,-121.467207,...,Active,2750,Standard,2024-12-23T00:00:00.000Z,None,2024-12-15T00:00:00.000Z,2024-12-23T04:48:44.619Z,1,"{'2024-12-23': {'event': 'Rental Listing', 'pr...",NaN
